# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import copy

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/TS_cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,16.16,80,0,7.20,ZA,1699394519
1,1,kerikeri,-35.2268,173.9474,18.28,74,52,1.79,NZ,1699394519
2,2,kununurra,-15.7667,128.7333,25.99,69,1,0.00,AU,1699394520
3,3,sungai penuh,-2.0833,101.3833,18.46,98,100,0.39,ID,1699394520
4,4,ushuaia,-54.8000,-68.3000,6.81,53,75,4.63,AR,1699394314


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 0.75,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
#My ideal weather: 20 to 30 celsius, 30-60% Relative Humidity, and 2-4 Mph Wind Speed 

ideal_weather_df = city_data_df[(city_data_df['Max Temp'] > 20) & (city_data_df["Max Temp"] < 30) &
                             (city_data_df["Humidity"] > 30) & (city_data_df["Humidity"] < 60) &
                             (city_data_df["Wind Speed"] > 2) & (city_data_df["Wind Speed"] < 4)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

#Index to Zero
ideal_weather_df = ideal_weather_df.reset_index(drop=True)

# Display sample data
ideal_weather_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,22,mildura,-34.2000,142.1500,24.48,57,18,2.85,AU,1699394527
1,56,fort bragg,35.1390,-79.0060,24.32,46,4,2.11,US,1699394539
2,71,strathfieldsaye,-36.8000,144.3667,21.68,56,4,3.55,AU,1699394546
3,154,altinopolis,-21.0256,-47.3739,29.64,48,0,2.18,BR,1699394502
4,168,awjilah,29.1081,21.2869,22.40,39,0,2.29,LY,1699394573
5,224,hamilton,39.1834,-84.5333,22.69,37,0,2.57,US,1699394551
6,238,ar rahad,12.7167,30.6500,29.30,46,95,2.82,SD,1699394596
7,245,greenwood,34.1668,-82.1165,25.45,46,0,3.09,US,1699394510
8,285,kone,-21.0595,164.8658,25.68,50,6,2.46,NC,1699394625
9,347,sao francisco,-15.9486,-44.8644,28.14,44,58,2.27,BR,1699394645


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
0,mildura,AU,-34.2000,142.1500,57,
1,fort bragg,US,35.1390,-79.0060,46,
2,strathfieldsaye,AU,-36.8000,144.3667,56,
3,altinopolis,BR,-21.0256,-47.3739,48,
4,awjilah,LY,29.1081,21.2869,39,
5,hamilton,US,39.1834,-84.5333,37,
6,ar rahad,SD,12.7167,30.6500,46,
7,greenwood,US,34.1668,-82.1165,46,
8,kone,NC,-21.0595,164.8658,50,
9,sao francisco,BR,-15.9486,-44.8644,44,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    #break the API
    #print(name_address)
    #break
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mildura - nearest hotel: Chaffey Motor Inn
fort bragg - nearest hotel: Airborne Inn Lodging
strathfieldsaye - nearest hotel: Quality Resort All Seasons
altinopolis - nearest hotel: Vila das Palmeiras
awjilah - nearest hotel: No hotel found
hamilton - nearest hotel: North Vista Manor
ar rahad - nearest hotel: بيت عمر ٨٠٠ متر
greenwood - nearest hotel: Hospitality Suites
kone - nearest hotel: Pacifik Appartels
sao francisco - nearest hotel: Hotel Atalaia
poquoson - nearest hotel: Bayview Towers
bastrop - nearest hotel: No hotel found
rajpur - nearest hotel: No hotel found
al majaridah - nearest hotel: No hotel found
port augusta - nearest hotel: Hotel Commonwealth


,City,Country,Lat,Lng,Humidity,Hotel Name
0,mildura,AU,-34.2000,142.1500,57,Chaffey Motor Inn
1,fort bragg,US,35.1390,-79.0060,46,Airborne Inn Lodging
2,strathfieldsaye,AU,-36.8000,144.3667,56,Quality Resort All Seasons
3,altinopolis,BR,-21.0256,-47.3739,48,Vila das Palmeiras
4,awjilah,LY,29.1081,21.2869,39,No hotel found
5,hamilton,US,39.1834,-84.5333,37,North Vista Manor
6,ar rahad,SD,12.7167,30.6500,46,بيت عمر ٨٠٠ متر
7,greenwood,US,34.1668,-82.1165,46,Hospitality Suites
8,kone,NC,-21.0595,164.8658,50,Pacifik Appartels
9,sao francisco,BR,-15.9486,-44.8644,44,Hotel Atalaia


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display
#Hover Cols
hover = ['City', 'Humidity', 'Hotel Name']

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriNatGeo",
    frame_width = 500,
    frame_height = 300,
    size = "Humidity",
    scale = 0.75,
    color = "City",
    hover_cols = hover
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name)